In [ ]:
import os
import vector
import mplhep
import numpy as np
import pandas as pd
import awkward as ak
import boost_histogram as bh
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep
import gc
mplhep.style.use("CMS")

In [ ]:
qq_test = ak.from_parquet(r"qq_train.parquet")
z_test = ak.from_parquet(r"z_train.parquet")
zh_test = ak.from_parquet(r"zh_train.parquet")

In [ ]:
# Create 4-vectors from the information in the .parquet file test and access information such as energy or pT:

def to_p4(p4_obj):
    return vector.awk(
        ak.zip(
            {
                "mass": p4_obj.tau,
                "x": p4_obj.x,
                "y": p4_obj.y,
                "z": p4_obj.z,
            }
        )
    )
def my_to_ptphietaenergy(p4_obj):
    return vector.awk(
        ak.zip(
            {
                "pt": p4_obj.pt,
                "phi": p4_obj.phi,
                "eta": p4_obj.rapidity,
                "E": p4_obj.E
            }
        )
    )

def to_bh(test, bins, cumulative=False):
    h1 = bh.Histogram(bh.axis.Variable(bins))
    h1.fill(test)
    if cumulative:
        h1[:] = np.sum(h1.values()) - np.cumsum(h1)
    return h1

# ZH
zh_gen_tau_p4s = to_p4(zh_test.gen_jet_tau_p4s)
zh_reco_cand_p4s = to_p4(zh_test.reco_cand_p4s)
zh_reco_jet_p4s = to_p4(zh_test.reco_jet_p4s)

zh_test["4momentum"] = my_to_ptphietaenergy(zh_reco_cand_p4s)
# Z
z_gen_tau_p4s = to_p4(z_test.gen_jet_tau_p4s)
z_reco_cand_p4s = to_p4(z_test.reco_cand_p4s)
z_reco_jet_p4s = to_p4(z_test.reco_jet_p4s)
z_gen_tau_pt = z_gen_tau_p4s.pt
z_test["4momentum"] = my_to_ptphietaenergy(z_reco_cand_p4s)
#qq
qq_test_reco_cand_p4s = to_p4(qq_test["reco_cand_p4s"])
qq_test_reco_jet_p4s = to_p4(qq_test["reco_jet_p4s"])
qq_test["4momentum"] = my_to_ptphietaenergy(qq_test_reco_jet_p4s)

In [ ]:
ak.to_parquet(qq_test, "my_qqcand_test.parquet")
ak.to_parquet(z_test, "my_zcand_test.parquet")
ak.to_parquet(zh_test, "my_zhcand_test.parquet")